In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd, numpy as np, seaborn as sb, matplotlib.pyplot as plt
import matplotlib 
matplotlib.rcParams["figure.figsize"] = (20,10)

In [ ]:
df = pd.read_csv("../input/bengaluru-house-price-data/Bengaluru_House_Data.csv")
df

#dependent_varaible = df["price"]

In [ ]:
df.shape

In [ ]:
x=df.groupby("area_type")["area_type"].agg("count")
x

In [ ]:
plt.xlabel("area_type")
plt.plot(x)

In [ ]:
x=df.groupby(["area_type"]).agg("count")
x

In [ ]:
plt.plot(x)

In [ ]:
df1=df.drop(["availability","area_type","society","balcony"],axis=1)
df1.head()

In [ ]:
df1.isnull().sum()

In [ ]:
df2=df1.dropna()
df2.shape

In [ ]:
df2.isnull().sum()

In [ ]:
df2["size"].unique()

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df2["bhk"]=df2["size"].apply(lambda x: int(x.split(" ")[0]))
df2.head()

In [ ]:
df2["bhk"].unique()

In [ ]:
df2[df2["bhk"]>20]

#error in total_sqft 2400 cant have 43bhk seeems like an error

In [ ]:
df2["total_sqft"].unique()

#total_sqft has some rangess

In [ ]:
def is_float(x):
    try:
        float(x)
    except:
        return False
    return True

In [ ]:
df2[~df2["total_sqft"].apply(is_float)].head(10)

In [ ]:
def total_sqft_to_num(x):
    y=x.split("-")
    if len(y) == 2:
        return (float(y[0])+float(y[1]))/2
    try:
        return float(x)
    except:
        return None

In [ ]:
df3=df2.copy()
df3["total_sqft"]=df3["total_sqft"].apply(total_sqft_to_num)

In [ ]:
df3.head()

In [ ]:
df3["total_sqft"].values

In [ ]:
df3.dtypes

In [ ]:
df4=df3.copy()
df4["price_per_sqft"]=df4["price"]*100000/df4["total_sqft"]
df4.head()

In [ ]:
len(df4["location"].unique())

In [ ]:
df4["location"]=df4["location"].apply(lambda x:x.strip())
location_stats=df4.groupby("location")["location"].agg("count").sort_values(ascending=False)
location_stats

In [ ]:
len(location_stats[location_stats<=10])

In [ ]:
location_stats_less_than_10 = location_stats[location_stats<=10]
location_stats_less_than_10

In [ ]:
len(df4["location"].unique())

In [ ]:
df4["location"]=df4["location"].apply(lambda x: "other" if x in location_stats_less_than_10 else x)
len(df4["location"].unique())

In [ ]:
df4.head(10)

In [ ]:
df4[df4["total_sqft"]/df4["bhk"] < 300].head()

In [ ]:
df4.shape

In [ ]:
df5 = df4[~(df4["total_sqft"]/df4["bhk"] < 300)]
df5.shape

In [ ]:
df5.price_per_sqft.describe()

#here nearly 68% datapoints are mean and 1std and removing the others

In [ ]:
def remove_price_per_sqft_outliers(df):
    df_out = pd.DataFrame()
    for key, subdf in df.groupby('location'):
        m = np.mean(subdf.price_per_sqft)
        st = np.std(subdf.price_per_sqft)
        reduced_df = subdf[(subdf.price_per_sqft>(m-st)) & (subdf.price_per_sqft<=(m+st))]
        df_out = pd.concat([df_out,reduced_df],ignore_index=True)
    return df_out
df6 = remove_pps_outliers(df5)
df6.shape

In [ ]:
def plot_scatter_chart(df,location):
    bhk2 = df[(df.location==location) & (df.bhk==2)]
    bhk3 = df[(df.location==location) & (df.bhk==3)]
    matplotlib.rcParams['figure.figsize'] = (15,10)
    plt.scatter(bhk2.total_sqft,bhk2.price,color='blue',label='2 BHK', s=50)
    plt.scatter(bhk3.total_sqft,bhk3.price,marker='+', color='green',label='3 BHK', s=50)
    plt.xlabel("Total Square Feet Area")
    plt.ylabel("Price (Lakh Indian Rupees)")
    plt.title(location)
    plt.legend()
    
plot_scatter_chart(df6,"Rajaji Nagar")

In [ ]:

plot_scatter_chart(df6,"Hebbal")

In [ ]:
def remove_bhk_outliers(df):
    exclude_indices = np.array([])
    for location, location_df in df.groupby('location'):
        bhk_stats = {}
        for bhk, bhk_df in location_df.groupby('bhk'):
            bhk_stats[bhk] = {
                'mean': np.mean(bhk_df.price_per_sqft),
                'std': np.std(bhk_df.price_per_sqft),
                'count': bhk_df.shape[0]
            }
        for bhk, bhk_df in location_df.groupby('bhk'):
            stats = bhk_stats.get(bhk-1)
            if stats and stats['count']>5:
                exclude_indices = np.append(exclude_indices, bhk_df[bhk_df.price_per_sqft<(stats['mean'])].index.values)
    return df.drop(exclude_indices,axis='index')
df7 = remove_bhk_outliers(df6)
# df7 = df6.copy()
df7.shape

In [ ]:
plot_scatter_chart(df7,"Rajaji Nagar")


In [ ]:
plot_scatter_chart(df7,"Hebbal")


In [ ]:
import matplotlib
matplotlib.rcParams["figure.figsize"] = (20,10)
plt.hist(df7.price_per_sqft,rwidth=0.8)
plt.xlabel("Price Per Square Feet")
plt.ylabel("Count")


In [ ]:
df7.bath.unique()


In [ ]:
plt.hist(df7.bath,rwidth=0.8)
plt.xlabel("Number of bathrooms")
plt.ylabel("Count")

In [ ]:
df7[df7["bath"]>10]

In [ ]:
df7[df7.bath>df7.bhk+2]


In [ ]:
df8 = df7[df7.bath<df7.bhk+2]
df8.shape

In [ ]:
df8.head(2)


In [ ]:
df9 = df8.drop(['size','price_per_sqft'],axis='columns')
df9.head(3)

In [ ]:
dummies = pd.get_dummies(df9.location)
dummies.head(3)

In [ ]:
df10 = pd.concat([df9,dummies.drop('other',axis='columns')],axis='columns')
df10.head()

In [ ]:
df11 = df10.drop('location',axis='columns')
df11.head(2)

In [ ]:
df11.shape


In [ ]:
X = df11.drop(['price'],axis='columns')
X.head(3)


In [ ]:
X.shape


In [ ]:
y = df11.price
y.head(3)

In [ ]:
len(y)


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=10)

In [ ]:
from sklearn.linear_model import LinearRegression
lr_clf = LinearRegression()
lr_clf.fit(X_train,y_train)
lr_clf.score(X_test,y_test)

In [ ]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)

cross_val_score(LinearRegression(), X, y, cv=cv)

In [ ]:
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor

def find_best_model_using_gridsearchcv(X,y):
    algos = {
        'linear_regression' : {
            'model': LinearRegression(),
            'params': {
                'normalize': [True, False]
            }
        },
        'lasso': {
            'model': Lasso(),
            'params': {
                'alpha': [1,2],
                'selection': ['random', 'cyclic']
            }
        },
        'decision_tree': {
            'model': DecisionTreeRegressor(),
            'params': {
                'criterion' : ['mse','friedman_mse'],
                'splitter': ['best','random']
            }
        }
    }
    scores = []
    cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    for algo_name, config in algos.items():
        gs =  GridSearchCV(config['model'], config['params'], cv=cv, return_train_score=False)
        gs.fit(X,y)
        scores.append({
            'model': algo_name,
            'best_score': gs.best_score_,
            'best_params': gs.best_params_
        })

    return pd.DataFrame(scores,columns=['model','best_score','best_params'])

find_best_model_using_gridsearchcv(X,y)

In [ ]:
def predict_price(location,sqft,bath,bhk):    
    loc_index = np.where(X.columns==location)[0][0]

    x = np.zeros(len(X.columns))
    x[0] = sqft
    x[1] = bath
    x[2] = bhk
    if loc_index >= 0:
        x[loc_index] = 1

    return lr_clf.predict([x])[0]

In [ ]:
predict_price('1st Phase JP Nagar',1000, 2, 2)

In [ ]:
predict_price('1st Phase JP Nagar',1000, 3, 3)


In [ ]:
predict_price('Indira Nagar',1000, 2, 2)

In [ ]:
predict_price('Indira Nagar',1000, 3, 3)
